In [1]:
import glob
import os
import pprint 
# import pyyaml

import astropy.units as u
from IPython.display import display
import matplotlib as mp
import matplotlib.pylab as pl
import pandas as pd

import astropy.constants as constants

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
import astropy.units as u
from astropy.table import Table, vstack

from pystrometry.pystrometry import pjGet_a_barycentre
from astroquery.exoplanet_orbit_database import ExoplanetOrbitDatabase


In [2]:
LOCAL_PATH = os.path.dirname(os.getcwd())
TEMPORARY_DIR = os.path.join(LOCAL_PATH, 'temp')
if os.path.isdir(TEMPORARY_DIR) is False:
    os.makedirs(TEMPORARY_DIR)
print(LOCAL_PATH)

/Users/jsahlmann/code/github/JohannesSahlmann/pystrometry


In [3]:
pp = pprint.PrettyPrinter(indent=4)     
%matplotlib inline
mp.rcParams['figure.figsize'] = (18, 9)
mp.rcParams['font.size'] = 20
mp.rcParams['ytick.direction'] = 'in'
mp.rcParams['xtick.direction'] = 'in'

In [4]:
# mp.rcParams

In [5]:
# table

In [6]:
save_plot = True

In [7]:
from pystrometry.pystrometry import OrbitSystem 
from collections import OrderedDict

orb = OrbitSystem()  # default parameters
orb.plot_orbits() 

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
df = table.to_pandas()

In [ ]:

# if save_plot:
#     figure_file_name = os.path.join(TEMPORARY_DIR, '{}_astrometry_signatures.png'.format(selected_table_name))
#     pl.savefig(figure_file_name, transparent=False, bbox_inches='tight', pad_inches=0.05)        